## Introduction:

In this notebook, I'm gonna utilize the most basic classifier (Logistic Regression) to classify MNIST dataset using Theano library .. It is one of the most effective python libraries for implementing deep learning with the ability to run the codes on a GPU .. 

This implementation is steming from the following tutorial .. 
http://deeplearning.net/tutorial/deeplearning.pdf

In [479]:
from theano import *
import theano.tensor as T
import numpy as np
import timeit
import logistic_sgd
import gzip
import os
import sys
import six.moves.cPickle as pickle

In [461]:
logistic_sgd.sgd_optimization_mnist()

... loading data
... building the model
... training the model
epoch 1, minibatch 83/83, validation error 12.458333 %
     epoch 1, minibatch 83/83, test error of best model 12.375000 %
epoch 2, minibatch 83/83, validation error 11.010417 %
     epoch 2, minibatch 83/83, test error of best model 10.958333 %
epoch 3, minibatch 83/83, validation error 10.312500 %
     epoch 3, minibatch 83/83, test error of best model 10.312500 %
epoch 4, minibatch 83/83, validation error 9.875000 %
     epoch 4, minibatch 83/83, test error of best model 9.833333 %
epoch 5, minibatch 83/83, validation error 9.562500 %
     epoch 5, minibatch 83/83, test error of best model 9.479167 %
epoch 6, minibatch 83/83, validation error 9.322917 %
     epoch 6, minibatch 83/83, test error of best model 9.291667 %
epoch 7, minibatch 83/83, validation error 9.187500 %
     epoch 7, minibatch 83/83, test error of best model 9.000000 %
epoch 8, minibatch 83/83, validation error 8.989583 %
     epoch 8, minibatch 83/83,

The code for file logistic_sgd.pyc ran for 18.7s


Above is the call for the already exist implementation .. 

Let's try to write this code bit by bit for better understanding .. 

## Initialization:

In [462]:
## input is X .. while n_in is the number of nodes
class LogisticRegression(object):
    def _init_(self, input, n_in, n_out):
            ## init the weights
        self.W = theano.shared(value = np.zeros((n_in, n_out), dtype = theano.config.floatX), name = 'W', borrow = True)

            ## init the biases
        self.b = theano.shared(value=np.zeros((n_out,), dtype=theano.config.floatX), name='b', borrow=True)

            ## activition function 
        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W) + self.b)

            ## Prediction
        self.y_pred = T.argmax(self.p_y_given_x, axis=1) # each input will be assigned to the class that has the highest prob

            ## Parameters of the model are weights and biases..
        self.params = [self.W, self.b]

        self.input = input

        
    def negative_log_likelihood(self, y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
        
        
    def errors(self, y):
             # check if y has same dimension of y_pred
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
                )
            # check if y is of the correct datatype
        if y.dtype.startswith('int'):
            # the T.neq operator returns a vector of 0s and 1s, where 1
            # represents a mistake in prediction
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()

- p_y_given_x is a symbolic variable of vector-type .. resulted from the dot product of x and W and softmax activation function ..

- T.argmax returns the index at which p_y_given_x is maximal i.e. the class with maximum probability ..

In [374]:
## testing the function ..
input = [1, 20, 3, 4]
n_in = 4
n_out = 2

_init_(input, n_in, n_out)  ## the model doesn't do anything useful so far !

argmax

## Minimizing Loss Function:
The learning phase starts by minimizing the loss function .. it is very common to use the negative log-likelihood as the loss (i.e. maximizing the likelihood of the data set) ..

-ve Log Likelihood ==> Return the mean of the negative log-likelihood of the prediction
        of this model under a given target distribution.
        
the mean is chosen instead of the sum so that the learning rate is less dependent on the batch size

In [294]:
## this is a redundant implementation .. just to call the function and check it run correctly without errors ..
def negative_log_likelihood(y):
    return -T.mean(T.log(p_y_given_x)[T.arange(y.shape[0]), y])

errors() function ==> Return a float representing the number of errors in the minibatch
        over the total number of examples of the minibatch ; zero one
        loss over the size of the minibatch

In [257]:
## this is a redundant implementation .. just to call the function and check it run correctly without errors ..
def errors(y):
     # check if y has same dimension of y_pred
    if y.ndim != y_pred.ndim:
        raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', y_pred.type)
            )
        # check if y is of the correct datatype
    if y.dtype.startswith('int'):
            # the T.neq operator returns a vector of 0s and 1s, where 1
            # represents a mistake in prediction
        return T.mean(T.neq(y_pred, y))
    else:
        raise NotImplementedError()

## Loading the data:

Note that:

six.moves.cPickle is utilized here to provides simple utilities for wrapping over differences between Python 2 and Python 3. It is intended to support codebases that work on both Python 2 and 3 without modification. six consists of only one Python file, so it is painless to copy into a project.

In [463]:
def load_data(dataset):  ## input data is a string
    dataset = '/home/eman/PhD/Deep Learning Practice/Myown practice/mnist.pkl.gz'
    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)
#     return train_set, valid_set, test_set

    def shared_dataset(data_xy, borrow=True):
        """ Function that loads the dataset into shared variables

        The reason we store our dataset in shared variables is to allow
        Theano to copy it into the GPU memory (when code is run on GPU).
        Since copying data into the GPU is slow, copying a minibatch everytime
        is needed (the default behaviour if the data is not in a shared
        variable) would lead to a large decrease in performance.
        """
        data_x, data_y = data_xy
        shared_x = theano.shared(np.asarray(data_x,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        # When storing data on the GPU it has to be stored as floats
        # therefore we will store the labels as ``floatX`` as well
        # (``shared_y`` does exactly that). But during our computations
        # we need them as ints (we use labels as index, and if they are
        # floats it doesn't make sense) therefore instead of returning
        # ``shared_y`` we will have to cast it to int. This little hack
        # lets ous get around this issue
        return shared_x, T.cast(shared_y, 'int32')

    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval

## Sharing the data:

## Stochastic Gradient Descent:
For optimizing the variables of the classification model .. 

This function demonstrate stochastic gradient descent optimization of a log-linear
    model

In [483]:
def sgd_optimization_mnist():
    learning_rate = 0.13
    n_epochs = 1000
    dataset = 'mnist.pkl.gz'
    batch_size = 600
    this_validation_loss = 0
    
    datasets = load_data(dataset)
    
    #the dataset is divided into train, validation and test sets
    train_set_x, train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    
#     print train_set_x
    
    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] // batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] // batch_size

        
     ####################
    # BUILDING THE MODEL #
    ####################
    
    ## We have to allocate symbolic variables for the data, inputs and outputs .. 
    
    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch

    # generate symbolic variables for input (x and y represent a minibatch)
    x = T.matrix('x')  # data, presented as rasterized images
    y = T.ivector('y')  # labels, presented as 1D vector of [int] labels
    
    # Initialization .. Each MNIST image has size 28*28
    classifier = LogisticRegression() 
    classifier._init_(input=x, n_in=28 * 28, n_out=10) # each output corresponding to digit 
    
    # Define the cost function ..
    cost = classifier.negative_log_likelihood(y)

     ## testing the model 
    test_model = theano.function(
        inputs=[index],
        outputs = classifier.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        })
    
        
    ## Validate the model .. 
    validate_model = theano.function(
        inputs=[index],
        outputs = classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
  
        
#     # compute the gradient of cost with respect to theta = (W,b)
    g_W = T.grad(cost = cost, wrt = classifier.W)
    g_b = T.grad(cost = cost, wrt = classifier.b)
        
#     ## Model updates .. 
    updates = [(classifier.W, classifier.W - learning_rate * g_W),
               (classifier.b, classifier.b - learning_rate * g_b)]
     
#     ## Model  training .. 
    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    
#     ###############
#     # TRAIN MODEL #
#     ###############
        
    patience = 5000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is found
    improvement_threshold = 0.995  # a relative improvement of this much is considered significant
    validation_frequency = min(n_train_batches, patience//2)
#                                   # go through this many
#                                   # minibatche before checking the network
#                                   # on the validation set; in this case we
#                                   # check every epoch
        
    best_validation_loss = np.inf
    test_score = 0.
    start_time = timeit.default_timer()
        
    done_looping = False
    epoch = 0
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):
            minibatch_avg_cost = train_model(minibatch_index)  ## i.e. minibatch avg cost: 0.291112316698
            
#             # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
#                 # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i in range(n_valid_batches)]
                this_validation_loss = np.mean(validation_losses)  ## i.e. this valid_loss: 0.0720833333333
    
                print('epoch %i, minibatch %i/%i, validation error %f %%' %
                      (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )
     # if we got the best validation score until now
            if (this_validation_loss < best_validation_loss):
                #improve patience if loss improvement is good enough
                if this_validation_loss < best_validation_loss *  \
                    improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                        
                best_validation_loss = this_validation_loss
                    
            # test it on the test set

                test_losses = [test_model(i) for i in range(n_test_batches)]
                test_score = np.mean(test_losses)

                print(
                        (
                            ' epoch %i, minibatch %i/%i, test error of'
                            ' best model %f %%'
                        ) %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            test_score * 100.
                        )
                    )
        
#             # save the best model
            with open('best_model.pkl', 'wb') as f:
                pickle.dump(classifier, f)
        
        if patience <= iter:
            done_looping = True
            break

    end_time = timeit.default_timer()
    
    ## printting the results ..
    print((
            'Optimization complete with best validation score of %f %%,'
            'with test performance %f %%'
        )% (best_validation_loss * 100., test_score * 100.))
    print('The code run for %d epochs, with %f epochs/sec' % (
        epoch, 1. * epoch / (end_time - start_time)))

In [484]:
## Example .. test mnist dataset

def predict():
    
    # load the saved model
    classifier = pickle.load(open('best_model.pkl'))
    
     # compile a predictor function
    predict_model = theano.function(
        inputs=[classifier.input],
        outputs=classifier.y_pred)
    
    # We can test it on some examples from test test
    dataset='mnist.pkl.gz'
    datasets = load_data(dataset)
    test_set_x, test_set_y = datasets[2]
    test_set_x = test_set_x.get_value()
    
    predicted_values = predict_model(test_set_x[:10])
    print("Predicted values for the first 10 examples in test set:")
    print(predicted_values)

In [485]:
if __name__ == '__main__':
    sgd_optimization_mnist()

 epoch 1, minibatch 1/83, test error of best model 41.385417 %
epoch 1, minibatch 83/83, validation error 12.458333 %
epoch 2, minibatch 83/83, validation error 11.010417 %
epoch 3, minibatch 83/83, validation error 10.312500 %
epoch 4, minibatch 83/83, validation error 9.875000 %
epoch 5, minibatch 83/83, validation error 9.562500 %
epoch 6, minibatch 83/83, validation error 9.322917 %
epoch 7, minibatch 83/83, validation error 9.187500 %
epoch 8, minibatch 83/83, validation error 8.989583 %
epoch 9, minibatch 83/83, validation error 8.937500 %
epoch 10, minibatch 83/83, validation error 8.750000 %
epoch 11, minibatch 83/83, validation error 8.666667 %
epoch 12, minibatch 83/83, validation error 8.583333 %
epoch 13, minibatch 83/83, validation error 8.489583 %
epoch 14, minibatch 83/83, validation error 8.427083 %
epoch 15, minibatch 83/83, validation error 8.354167 %
epoch 16, minibatch 83/83, validation error 8.302083 %
epoch 17, minibatch 83/83, validation error 8.250000 %
epoch 18

## Comment:

On an Intel(R) Core(TM)2 Duo CPU E8400 @ 3.00 Ghz the code runs with approximately 1.936 epochs/sec
and it took 75 epochs to reach a test error of 7.489%. On the GPU the code does almost 10.0 epochs/sec.
For this instance we used a batch size of 600. 

However, on my laptop the code run for 61 epochs within 1.482160 eps/sec

This is the end of training MNIST dataset using simple logistic regression model and SGD .. The next step is to try MLP model and compare the results of both.